# Pollen Challenge ICPR 2020

## Segmentation

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['SM_FRAMEWORK'] = 'tf.keras'
import cv2
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import segmentation_models as sm
import albumentations as A
from sklearn.model_selection import train_test_split
import json
from tqdm import tqdm

In [ ]:
image_names = []
image_labels = []
image_masks = []
for i in range(1, 5):
    obj_dir = f'./train/images/{i}/train_OBJ/'
    mask_dir = f'./train/images/{i}/train_MASK/'
    images = os.listdir(obj_dir)
    for j in range(len(images)):
        if images[j].split('.')[-1] == 'png':
            image_names.append(obj_dir + images[j])
            image_labels.append(i)
            image_masks.append(mask_dir + images[j].replace('OBJ', 'MASK'))

In [ ]:
images = []
masks = []
for i in range(len(image_names)):
    if image_names[i].split('.')[-1] == 'png':
        image = cv2.imread(image_names[i])
        images.append(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        masks.append(cv2.imread(image_masks[i], 0))

### Training

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images, masks, test_size=0.1)

In [ ]:
def round_clip_0_1(x, **kwargs):
    return x.round().clip(0., 1.)

def get_training_augmentation():
    train_transform = [
        A.Resize(96, 96),
        A.HorizontalFlip(p=0.5),
        A.ShiftScaleRotate(scale_limit=0.5, rotate_limit=0, shift_limit=0.1, p=1, border_mode=0),
        A.IAAAdditiveGaussianNoise(p=0.2),
        A.IAAPerspective(p=0.5),
        A.OneOf(
            [
                A.CLAHE(p=1),
                A.RandomBrightness(p=1),
                A.RandomGamma(p=1),
            ],
            p=0.9,
        ),
        A.OneOf(
            [
                A.IAASharpen(p=1),
                A.Blur(blur_limit=3, p=1),
                A.MotionBlur(blur_limit=3, p=1),
            ],
            p=0.9,
        ),
        A.OneOf(
            [
                A.RandomContrast(p=1),
                A.HueSaturationValue(p=1),
            ],
            p=0.9,
        ),
        A.Lambda(mask=round_clip_0_1)
    ]
    return A.Compose(train_transform)

def get_validation_augmentation():
    train_transform = [
        A.Resize(96, 96),
        A.Lambda(mask=round_clip_0_1)
    ]
    return A.Compose(train_transform)

In [ ]:
for i in range(len(X_train)):
    result = get_training_augmentation()(image=X_train[i], mask=y_train[i])
    X_train[i] = result['image']
    y_train[i] = result['mask']
for i in range(len(X_val)):
    result = get_validation_augmentation()(image=X_val[i], mask=y_val[i])
    X_val[i] = result['image']
    y_val[i] = result['mask']

In [ ]:
del images
del masks
del image_names
del image_labels
del image_masks

In [ ]:
BACKBONE = 'resnet34'
BATCH_SIZE = 64
CLASSES = ['segment']
LR = 0.0001
EPOCHS = 40

preprocess_input = sm.get_preprocessing(BACKBONE)

In [ ]:
def get_preprocessing(preprocessing_fn):
    _transform = [
        A.Lambda(image=preprocessing_fn),
    ]
    return A.Compose(_transform)

In [ ]:
for i in range(len(X_train)):
    result = get_preprocessing(preprocess_input)(image=X_train[i], mask=y_train[i])
    X_train[i] = result['image']
    y_train[i] = result['mask']

In [ ]:
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train)
X_val = np.asarray(X_val).astype(np.float32)
y_val = np.asarray(y_val)

In [ ]:
X_train = X_train / 255.
X_val = X_val / 255.

In [ ]:
model = sm.Unet(BACKBONE, classes=1, activation='sigmoid', input_shape=(96, 96, 3))

In [ ]:
optim = keras.optimizers.Adam(LR)

dice_loss = sm.losses.DiceLoss()
focal_loss = sm.losses.BinaryFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

model.compile(optim, total_loss, metrics)

In [ ]:
callbacks = [
    keras.callbacks.ModelCheckpoint('./segmentation_96_normalised.h5', save_weights_only=True, save_best_only=True, mode='min'),
    keras.callbacks.ReduceLROnPlateau(),
]

In [ ]:
history = model.fit(
    x=X_train,
    y=y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=callbacks,
    validation_data=(X_val, y_val)
)

### Validation – with classifier

In [ ]:
clf = keras.models.load_model('vgg16_segmented_96_noaug_lastconvtop_10.h5')

In [ ]:
model.load_weights('segmentation_96_normalised.h5')

In [ ]:
X_val_seg = model.predict(X_val)

In [ ]:
X_val_seg_round = X_val_seg > 0.5

In [ ]:
X_val_seg_round = X_val_seg_round.astype(np.uint8)

In [ ]:
images = []
for i in tqdm(range(len(X_val))):
    img = X_val[i]
    res = cv2.bitwise_and(img, img, mask=X_val_seg_round[i].reshape((96, 96)))
    images.append(res)

In [ ]:
images = np.array(images)

In [ ]:
np.save('segmented_val.npy', images)

In [ ]:
y_val_pred = clf.predict(images)

In [ ]:
y_val_pred = np.argmax(y_val_pred, axis=1)

### Testing

In [ ]:
test_image_names = []
test_input_dir = f"./test/images/"
test_images = os.listdir(test_input_dir)
for j in range(len(test_images)):
    if test_images[j].split(".")[-1] == "png":
        test_image_names.append(test_input_dir + test_images[j])

In [ ]:
test_images = []
for test_image in test_image_names:
    if test_image.split('.')[-1] == 'png':
        test_images.append(cv2.resize(cv2.imread(test_image, cv2.IMREAD_UNCHANGED),
                                      (96, 96), interpolation=cv2.INTER_AREA))

In [ ]:
test_images = np.array(test_images).astype(np.float32)
test_images = test_images / 255.

In [ ]:
X_test_seg = model.predict(test_images)

In [ ]:
X_test_seg_round = X_test_seg > 0.5

In [ ]:
X_test_seg_round = X_test_seg_round.astype(np.uint8)

In [ ]:
images = []
for i in tqdm(range(len(test_images))):
    img = test_images[i]
    res = cv2.bitwise_and(img, img, mask=X_test_seg_round[i].reshape((96, 96)))
    images.append(res)

In [ ]:
images = np.array(images)

In [ ]:
plt.imshow(images[0])

In [ ]:
test_pred = np.argmax(clf.predict(images), axis=1)

In [ ]:
test_pred

In [ ]:
pd.value_counts(test_pred)

In [ ]:
submission = []
for i in range(len(test_image_names)):
    submission.append({"Filename": "{}".format(test_image_names[i].split("/")[-1]),
                       "Class": str(test_pred[i] + 1)
                      })

In [ ]:
with open('submission_segmented_trainonly_vgg16_96.json', 'w', encoding='utf-8') as f:
    json.dump(submission, f, ensure_ascii=False, indent=4)